In [ ]:
%%html
<link rel="stylesheet" href="static/hyrule.css" type="text/css">

In [ ]:
%pylab inline

In [3]:
# pylab inline (either in the notebook or as an argument starting the notebook)
# will by default load numpy as np and matplotlib.pylab as plt
# Let's import everything else we used last time as well:

import pandas as pd
import seaborn as sns
from __future__ import division
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.options.display.mpl_style = 'default'

/Users/joshleeman/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2881: FutureWarning: 
mpl_style had been deprecated and will be removed in a future version.
Use `matplotlib.pyplot.style.use` instead.

  exec(code_obj, self.user_global_ns, self.user_ns)


# Data Exploration and Visualisation

## Objectives

* learn how to group and aggregate data in pandas
* learn how to visualise data with pandas
* explain the difference between cast and melt in terms of a data frame
* explore common visual relationships in data
* learn two visualisation packages in python independent of pandas

## Code Dictionary

### Additional Pandas Code

Function         | Description
-----------------|------------
`.groupby()`     | Creates a groupby object in pandas. pass in either a string (one column) or a list (multiple columns)
`.agg()`         | Aggregates data into a new DataFrame from a groupby object
`.pivot_table()` | generates a pivot table, akin to those you'd expect in excel.
`melt()`      | melts a DataFrame
`.plot()`        | general plot using matplotlib of the columns provided, using the index as the x-axis
`.hist()`        | generates a histogram using matplotlib of the columns provided, using the index as the x-axis
`.boxplot()`     | generates a boxplot, which displays distribution via quartiles and outliers
`pd.tools.plotting.scatter_matrix()` | a representation of histograms and scatterplots between columns <br /> Very common to import seperately using from pd.tools.plotting import scatter_matrix

<br />
### Matplotlib Basics

Run this first to generate some data! They'll relate to the matplotlib basics below.

```py
X = np.linspace(-np.pi, np.pi, 256, endpoint=True)
C, S = np.cos(X), np.sin(X)
```

Matplotlib functions will start with `plt`. In science we work with the concept of a `figure`, which is the space in which plots exist. While the below code is for `plot`, others are common as well (such as `hist`)

Code | Description
-----|------------
`plt.figure(figsize=(8, 6), dpi=80)` | Sets a figure size of 8 by 6 inches, with a dpi of 80
`plt.subplot(1, 1, 1)` | Create a new subplot from a grid of 1x1
`plt.plot(X, C, color="blue", linewidth=1.0, linestyle="-")` | Plots cosine with a blue continuous line of width 1 (pixels)
`plt.plot(X, S, color="green", linewidth=1.0, linestyle="-")` | Plots sine with a green continuous line of width 1 (pixels)
`plt.xlim(-4.0, 4.0)` | Set x limits
`plt.xticks(np.linspace(-4, 4, 9, endpoint=True))` | set x ticks
`plt.ylim(-1.0, 1.0)` | Set y limits
`plt.yticks(np.linspace(-1, 1, 5, endpoint=True))` | Set y ticks
`plt.savefig("exercice_2.png", dpi=72)` | Save figure using 72 dots per inch
`plt.show()` | Show result on screen

### Seaborn Basics

We'll need to install seaborn first. You can do this with conda, a package installer for Anaconda. With this, conda might also update other packages. seaborn, once imported, takes over default matplotlib figures

```sh
cd ~/anaconda/bin
conda install seaborn
```

Code | Description
-----|------------
`.set_style()`  | changes the current seaborn style. pass in parameters using a dictionary to update settings.
`.axes_style()` | returns a dictionary of the current styles settings 
`.despine()`    | removes the top and right ticks on a chart

## Lecture Notes

### Exploring the Wine Data Set

wine data set: https://archive.ics.uci.edu/ml/machine-learning-databases/wine/

### Aggregating Data

Last class was heavily focused on manipulating and subsetting individual observations in a data set. Exploring the raw data can be benefited by exploring the data as a whole through grouping and aggregations. Data aggregations and segmentations allow us to better understand and evaluate the properties of our data, particularly when we have a target variable in mind.

In [ ]:
columns = """-- The attributes are (dontated by Riccardo Leardi, 
    riclea@anchem.unige.it )
    1) Alcohol
    2) Malic acid
    3) Ash
    4) Alcalinity of ash  
    5) Magnesium
    6) Total phenols
    7) Flavanoids
    8) Nonflavanoid phenols
    9) Proanthocyanins
    10)Color intensity
    11)Hue
    12)OD280/OD315 of diluted wines
    13)Proline"""

# we can generate the columns list using some string and list functions.
columns = columns.split('\n')
columns = [i.strip() for i in columns][2:]
columns = [i.split(')')[1].strip().lower() for i in columns]
cols = ['class']
cols.extend(columns)
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/wine/wine.data'
wine = pd.read_csv(url, header=None)
wine.columns = cols
wine.head()

In [ ]:
# Create a groupby object:
winegroups = wine.groupby('class')
print type(winegroups)

# groupby objects allow for some standard pandas calls:

winegroups.describe()

In [ ]:
winegroups.sum()

In [ ]:
# Groupby objects also:
## can work through selected columns

#print winegroups.alcohol.mean()
#print winegroups[['alcohol', 'ash']].median()

## can work through aggregation dictionaries, which can reference the origin
## groupby functions, or construct new functions.

#print winegroups.agg({'alcohol': 'median'})
print winegroups.agg({'alcohol': lambda alcohol: alcohol.nunique() / alcohol.count()})


### Pivot Tables

Pivot tables are another aggregational tool provided in pandas. Use pivot tables when:

1. we want to easily compare results of variables across groups
2. we want to easily plot results along the index.


In [ ]:
pivoted_wine = wine.pivot_table(values='ash',
                 columns='class',
                 index='alcohol',
                 aggfunc='count')

### Aggregation shapes

Sometimes we'll need to "melt" or "cast" our aggregation into a different shape, mostly in the context of plotting or applying aggregations as new fields in the original data set. Consider the use case of `melt()` here.


In [ ]:
pivoted_wine_reset = pivoted_wine.reset_index()
print pivoted_wine_reset.head()

print pd.melt(pivoted_wine_reset,
              id_vars=['alcohol'],
              value_vars=[1, 2, 3],
              value_name='count').dropna().head(20)

### Aggregation Practice

1. This time, create a group by object on a rounded 'alcohol' field. You might want to break this out into:
    * create a new column with the rounded alcohol content
    * groupby on the new column
2. Explore the statistics from `.describe()` in this new groupby object. Across the new groups, what is interesting? What isn't?
3. Use a pivot table to show the count of each target type per rounded alcohol level.
4. Finally, melt the pivot table so that the columns represent each grouping and a count of that group.
5. **take a step back** What would be the `groupby()` code to create the same data frame you created in question 4?

In [ ]:
x = wine.alcohol.round()
wine.append(x)
alc_round = wine.groupby(x)
alc_round.describe()

### Baseball Stats

We'll pull in the Lahman batting stats for data visualisation. There's only one table in the Lahman database for now; we'll add more as class moves forward. As mentioned before, pandas can read in from databases as well, using a db engine /connection.

Below includes the columns and their meanings:
```
player         Player ID code
year           Year
stint          player's stint (order of appearances within a season)
team           Team
lgID           League
G              Games
AB             At Bats
R              Runs
H              Hits
X2B            Doubles
X3B            Triples
HR             Homeruns
RBI            Runs Batted In
SB             Stolen Bases
CS             Caught Stealing
BB             Base on Balls
SO             Strikeouts
IBB            Intentional walks
HBP            Hit by pitch
SH             Sacrifice hits
SF             Sacrifice flies
GIDP           Grounded into double plays
```

In [ ]:
import sqlite3
con = sqlite3.connect("../data/lahman.sqlite")
df = pd.read_sql("SELECT * from batting", con)
df = df.convert_objects(convert_numeric=True)

# Reminder: Many functions from data frames will work on groupby objects!
# What is below doing?
df.groupby('year').head(20)

### Single Variable Plots: Histograms and Boxplots

To best understand a feature on its own, we explore it using histograms. Histograms help us understand the distribution of data.

#### What shouild we be looking for?

1. Width of the data. Use this alongside standard deviation to understand scope.
2. Distributions. NYU has a [nice guideline](http://pages.stern.nyu.edu/~adamodar/New_Home_Page/StatFile/statdistns.htm) to consider. We'll go over distributions as they arise. But in particular:
3. Expected vs Actual: Is this the shape we expected? Does it make sense given the feature? 
4. Segments: What jumps out as interesting? For example, are we interested in observing an 80/20 problem, or interested in the majority?

#### Ways we generate histograms:

1. `df.col.hist()` : pandas integrates with matplotlib to build histograms on the fly. Pass in multiple columns to generate multiple histograms at the same time.
2. `plt.hist()`: direct matplotlib code.

#### Ways we generate boxplots:

1. `df.col.boxplot()` : pandas integration for boxplots. Pass in multiple columns to generate multiple boxplots at the same time.
2. `plt.boxplot()`: direct matplotlib code.

In [ ]:
## Histograms
# pandas style
df[['g', 'ab']].hist()

# matplotlib style
#sns.set_style('white') # sets seaborn "white" to the default matplotlib style
#plt.figure(figsize=(8, 6), dpi=80)
#plt.subplot(1, 1, 1)
#plt.hist(df.g, bins=20)

## Boxplots
# pandas style
#plt.figure(figsize=(8, 6), dpi=80)
#df[['g', 'ab']].boxplot()
#
## matplotlib style
#plt.figure(figsize=(8, 6), dpi=80)
#plt.subplot(1, 1, 1)
#plt.boxplot(df.g)

### Doing histograms faster

1. How do we use a for loop to generate a plot of each numeric field? Consider checking the fields `dtype` before making the plots
2. Wrap your histogram three-liner as a function. We should be in the habit of always generating figures and subplots, but it's less necessary to have to write all code each time you want to do so.

### Multivariable plots: Scatterplots

While histograms allow us to explore fields individually, scatter plots allow us to compare and correlate data points. 

#### What to look for

1. Degrees of correlation (+/-). Are the two features positively or negatively correlated? What's the impact of one on the other?
2. Type of relationship. Is the relationship linear? Polynomial? Logarithmic? 

#### Ways we generate scatter plots:

1. `df.plot(type='scatter')` : pandas integrates with matplotlib to build scatter plots on the fly.
2. `plt.plot()`: direct matplotlib code. You'll have to specify the marker.
2. `plt.scatter()`: simpler context for matplotlib

In [ ]:
last_year = df[df.year == 2014]
sns.set_style('whitegrid')

# using plot and the '.' notation.
plt.figure(figsize=(8, 6), dpi=80)
plt.subplot(1, 1, 1)
plt.plot(last_year.ab, last_year.h, '.')

# using scatter, and setting additional parameters.
plt.figure(figsize=(8, 6), dpi=80)
plt.subplot(1, 1, 1)
plt.scatter(last_year.ab, last_year.h, s=last_year.hr**2, alpha=0.3, c='g')

### Other useful pandas plots

#### Trellis Plotting

Trellis plotting is a technique to group histograms or scatterplots based on a segmentation. This will be familiar to R users (the `lattice` package adds this functionality, as well as ggplot's `facet_grid`). Trellis plotting helps us easily identify how a feature can change dependent on another. Trellis plotting is built into pandas.

In [ ]:
import pandas.tools.rplot as rplot
    
plt.figure()
plot = rplot.RPlot(last_year, x='hr')
plot.add(rplot.TrellisGrid(['lg','.']))
plot.add(rplot.GeomHistogram())
print plot.render(plt.gcf())

A similar output can be done using the "by" argument in the plotting syntax (This would be more akin to `facet_wrap` in ggplot)

In [ ]:
plt.figure()
df[df.stint < 5].hist('hr', by='stint')

plt.figure()
df[df.stint < 5].boxplot('hr', by='stint')

#### Scatter Matrix

While doing this through each variable could be conveniently handled in a for loop or using a trellis plot, pandas includes a scatter_matrix which will provide both a histogram of each numeric field, but also a scatter plot for each field comparison. Use this wisely, as it's rather heavy plotting, and therefore, dependent on your data, memory intensive. You can, of course, subset to columns you are interested in comparing.

In [ ]:
cols = ['ab', 'h', 'X2b', 'X3b', 'hr',]
pd.scatter_matrix(last_year[cols], figsize=(12, 12))

### Groupbys and grouped plotting

One final technique to consider (for today) is how we can use groupby and plotting alongside each other.

In [ ]:
years_grouped = df.groupby('year')

plt.figure()
years_grouped.hr.agg(['mean']).plot()

plt.figure()
years_grouped.hr.agg(['sum']).plot()

## On Your Own

A very famous data set called Fisher's Iris data is included in sklearn. Import it with the following code into pandas (otherwise, it's a numpy ndarray)

```python
from sklearn import datasets
iris = datasets.load_iris()
iris_description = iris.DESCR
irisdf = pd.DataFrame(iris.data, columns=iris.feature_names)
irisdf['target'] = iris.target
print iris_description
```

1. describe, aggregate, and visualise the iris data set. what's interesting about it?
    * be able to show both scatterplots and histograms using matplotlib, Trellis, and the scatter_matrix. Use the class target to group by.
2. Which flowers are similar? Which seems to be in "its own world?"
3. Write an .apply() function that would "human learn" the three flowers. It would be your best attempt to:
    * split the flowers apart using the four columns
    * assign either a 1, 2 or 3, similar to target
    * test the performance of accuracy by measure how often your results match the original target vs the total count


## Review, Next Steps, Reading

#### Assignment 2: Due on Monday, Feb 9th
* You'll be practicing this week's lectures in more detail with citibike data. Use the following [link](https://github.com/podopie/DAT18NYC/blob/master/assignments/02-data_manipulation_and_visualization.md) to get to the details.

#### Additional Resources

* Watch Andrew Montalenti's ["Rapid Data Visualization"](http://vimeo.com/79580138)
   * Includes Pandas; D3.js; NVD3.js; Vega / Vincent; and PhantomJS. 
* View Wickham's [Grammar of Graphics Slides](http://www.slideshare.net/hadley/grammar-of-graphics-past-present-future)
    * Or, read his [paper](http://vita.had.co.nz/papers/layered-grammar.html)
* [another matplotlib tutorial](http://jakevdp.github.io/mpl_tutorial/)
* [Principles of Analytic Graphics](https://github.com/DataScienceSpecialization/courses/blob/master/04_ExploratoryAnalysis/Principles/PrinciplesofAnalyticGraphics.pdf?raw=true)
* [Matplotlib Docs](http://matplotlib.org/users/pyplot_tutorial.html)
* [Pandas Docs](http://pandas.pydata.org/pandas-docs/stable/visualization.html)
* [Choosing a Good Chart](http://extremepresentation.typepad.com/.shared/image.html?/photos/uncategorized/choosing_a_good_chart.jpg)
* [Columbia's deck on visualisation](http://www2.research.att.com/~volinsky/DataMining/Columbia2011/Slides/Topic2-EDAViz.ppt)
* [More practice with matplotlib and customisation](http://nbviewer.ipython.org/github/fonnesbeck/Bios366/blob/master/notebooks/Section2_4-Matplotlib.ipynb)

#### Next Steps
* We'll be learning and using statsmodels next week as a tool for further exploring and explaining data relationships. Consider going through [the documentation](http://statsmodels.sourceforge.net/stable/index.html) in your spare time to learn more.